### Информация
- **Автор**: Зитцер Данил
- **Дата создания**: 9 мая 2021 года, 16:09:13
- **Краткое описание**: Распознавание фигур с помощью API, упакованного в контейнер.
- **Полезные ссылки**:
    - [Using OpenCV in Docker](https://stackoverflow.com/questions/55313610/importerror-libgl-so-1-cannot-open-shared-object-file-no-such-file-or-directo)
- **Выводы**:

In [1]:
import cv2
import numpy as np
import requests

from nptyping import NDArray
from pydantic import BaseModel, PositiveInt, validator
from typing import Any, Dict, List, Tuple, Union 

from IPython.display import clear_output

### Constants

In [2]:
IMAGE_H = 500
IMAGE_W = 500

### Shape generator

In [3]:
class Painter(BaseModel):
    image_h: PositiveInt
    image_w: PositiveInt
        
    @property
    def imsize(self) -> NDArray[(2, ), np.uint32]:
        return np.array([self.image_h, self.image_w], dtype=np.uint32)
    
    @property
    def imshape(self) -> NDArray[(3, ), np.uint32]:
        return np.array([self.image_h, self.image_w, 3], dtype=np.uint32)
    
    @property
    def random_color(self) -> NDArray[(3, ), np.uint8]:
        return np.random.randint(low=0, high=255, size=3, dtype=np.uint8).tolist()
    
    @property
    def black_background(self) -> NDArray[(Any, Any, 3), np.uint8]:
        return np.zeros(shape=self.imshape, dtype=np.uint8)
    
    def draw_random_square(self) -> NDArray[(Any, Any, 3), np.uint8]:
        p1 = np.random.randint(low=0, high=self.imsize.min(), size=2)
        p2 = p1 + np.random.randint(low=0, high=(self.imsize - p1).min())

        return cv2.rectangle(self.black_background, tuple(p1), tuple(p2), self.random_color, -1)

    def draw_random_circle(self) -> NDArray[(Any, Any, 3), np.uint8]:
        radius = np.random.randint(low=0, high=self.imsize.min() // 2 - 1)
        center = np.random.randint(low=0+radius, high=self.imsize.min()-radius, size=2)

        return cv2.circle(self.black_background, tuple(center), radius, self.random_color, -1)
    
    def draw_random_shape(self) -> NDArray[(Any, Any, 3), np.uint8]:
        return self.draw_random_circle() if np.random.rand() > 0.5 else self.draw_random_square()

### NPImage 

In [4]:
class NPImage(BaseModel):
    image: Union[List, np.ndarray]
        
    @validator('image')
    def image_validator(cls, v):
        if isinstance(v, list):
            v = np.array(v, dtype=np.uint8)
        if not isinstance(v, np.ndarray):
            raise TypeError('must be a numpy.ndarray')
        if v.dtype != np.uint8:
            raise TypeError('numpy.ndarray must have dtype equal numpy.uint8')
        if len(v.shape) != 3:
            raise ValueError('numpy.ndarray must have 3 dimension')
        if v.shape[2] != 3:
            raise ValueError('numpy.ndarray must have 3 channels')
        return v
        
    class Config:
        arbitrary_types_allowed = True
        
        json_encoders = {
            np.ndarray: lambda x: x.tolist()
        }

### Shape recognition by API 

In [5]:
SHAPE_RECOGNITION_API = 'http://127.0.0.1:5000/predictor/'

In [7]:
painter = Painter(image_h=IMAGE_H, image_w=IMAGE_W)

while True:
    image = painter.draw_random_shape()
    
    response = requests.post(
        url=SHAPE_RECOGNITION_API,
        data=NPImage(image=image).json()
    )
    
    if response.status_code == 200:
    
        print(response.json()['shape'])

        cv2.imshow('image', image)

        if cv2.waitKey(3000) & 0xFF == ord('q'):
            break

        clear_output(True)
    
cv2.destroyAllWindows()

circle
